In [ ]:
import os
os.chdir("../")
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pinecone import Pinecone, ServerlessSpec 





In [ ]:

# Load the PDF
pdf_path = "C:/Users/Sarthak/Downloads/harrison’s-principles-of-internal-medicine-21st-edition.pdf"
loader = PyPDFLoader(pdf_path)

# Extract data (list of Document objects)
extracted_data = loader.load()

# Split the extracted documents into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# Apply text splitting
text_chunks = text_split(extracted_data)

# Print a few chunks to verify
print(text_chunks[:2])  # Print first two chunks


In [ ]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:

import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(PINECONE_API_KEY)  # Ensure it's set correctly

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define Index Name
index_name = "medbot"


# Check if Index Exists
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Created new index: {index_name}")
else:
    print(f"Index '{index_name}' already exists.")

##Dont Run The Below Code

In [ ]:



from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [ ]:

# Example usage
embeddings = download_hugging_face_embeddings()
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
from dotenv import load_dotenv
load_dotenv()


In [ ]:

import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
print(PINECONE_API_KEY)  # Ensure it's set correctly




In [ ]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medbot",
    embedding=embeddings,
)


In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.4, max_tokens=100)



In [ ]:


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


Prompt Generation

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import speech_recognition as sr
from deep_translator import GoogleTranslator  

# ------------------------- Firebase Setup -------------------------
def initialize_firebase():
    cred = credentials.Certificate("C:/Users/Sarthak/OneDrive/Desktop/BITS/service_account_key.json")

    if not firebase_admin._apps:
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://smart-box-4da35-default-rtdb.asia-southeast1.firebasedatabase.app'
        })

def get_patient_data():
    ref = db.reference('patient')
    data = ref.get()
    return data.get('SOS', 'No SOS Message'), data.get('avgHeartRate', 'No Heart Rate Data'), data.get('avgTemperature', 'No Temperature Data')

# ------------------------- Speech Recognition -------------------------
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 बोलें... (Speak something)")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)

    try:
        detected_text = recognizer.recognize_google(audio, language="hi")  
        print(f"🗣 पहचाना गया: {detected_text}")
        return detected_text
    except sr.UnknownValueError:
        print("❌ आवाज़ समझ नहीं आई (Could not understand audio)")
        return None
    except sr.RequestError:
        print("⚠ सर्वर से समस्या (Speech recognition service error)")
        return None

# ------------------------- Translation -------------------------
def translate_text(text):
    translator = GoogleTranslator(source="auto", target="en")
    translation = translator.translate(text)  
    print(f"✅ Translated: {translation}")
    return translation  

# ------------------------- Main Function -------------------------
if __name__ == "__main__":
    initialize_firebase()

    choice = input("Enter 1 for Chat, 2 for Voice: ").strip()
    
    if choice == "1":
        user_input = input("📝 Type your query: ")
    elif choice == "2":
        speech_text = recognize_speech()
        user_input = translate_text(speech_text) if speech_text else "No speech input provided"
    else:
        print("❌ Invalid choice. Defaulting to Chat input.")
        user_input = input("📝 Type your query: ")

    sos, heart_rate, temperature = get_patient_data()
    final_prompt = f"{user_input} what immediate medications should be taken? and my Heart Rate is {heart_rate} bpm and body temperature {temperature} °C "

    print("\n📝 Generated Chatbot Prompt:\n", final_prompt)

response = rag_chain.invoke({"input": final_prompt })
print(response["answer"])


In [ ]:
response = rag_chain.invoke({"input": final_prompt })
print(response["answer"])


In [ ]:
from twilio.rest import Client
import random
import os
from dotenv import load_dotenv
load_dotenv()



twilio_sid = os.getenv("TWILIO_ACCOUNT_SID")
twilio_token = os.getenv("TWILIO_AUTH_TOKEN")

client = Client(twilio_sid, twilio_token)

message = client.messages.create(

  body=f"A patient has the following issue:\n{translated_speech}\n His Collected Data:\n Heart rate : {heart_rate}\n Body Temperature : {temperature} °C \n suggested advice :  {response['answer']} ",
  from_='+18153840986',
  to='+918766049932'
)
